In [ ]:
import pandas as pd
import geopandas as gpd
from folium import Marker,DivIcon
import folium
from geopandas import GeoDataFrame

## Preprocessing

Setting up the map from GeoJON

In [53]:
data = "data/oslo_bydeler_befolkning_2024.geojson"
gdf_bydeler = gpd.read_file(data)
gdf_bydeler = gdf_bydeler.to_crs(epsg=25832)
bike_map = folium.Map([59.91273, 10.74609], zoom_start=11)
folium.GeoJson(gdf_bydeler).add_to(bike_map)

bike_map

##### Creating Geodataframes

In [54]:
# Reading the geojson file into a GeoDataFrame
arealer_gdf = gpd.read_file(data)
arealer_gdf.head()

,fid,bydelsnr,bynavn,bydel,befolkning_2024,geometry
0,1,030101,Oslo,Gamle Oslo,63721,"MULTIPOLYGON (((10.66087 59.88365, 10.66472 59..."
1,2,030102,Oslo,Grünerløkka,65532,"MULTIPOLYGON (((10.75378 59.91469, 10.75453 59..."
2,3,030103,Oslo,Sagene,47627,"MULTIPOLYGON (((10.77156 59.93095, 10.77135 59..."
3,4,030104,Oslo,St.Hanshaugen,41571,"MULTIPOLYGON (((10.75136 59.91923, 10.75158 59..."
4,5,030105,Oslo,Frogner,60727,"MULTIPOLYGON (((10.65562 59.89038, 10.65381 59..."


In [55]:
# Calling the station method and creating a GeoDataFrame
from bysykkel_api import get_all_stations
from credentials import credentials

_headers = {
    "Client-Identifier": credentials["user_id"]
}

response = get_all_stations(_headers)
stations = response['data']['stations']

In [56]:
stations_gdf = gpd.GeoDataFrame(stations)
print(stations_gdf.shape)
stations_gdf.head()

(251, 10)


,station_id,name,address,cross_street,lat,lon,is_virtual_station,capacity,station_area,rental_uris
0,5220,Blindern vgs,Sognsveien,Under broen,59.949881,10.730158,False,20,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5220', 'i..."
1,5217,Egertorget,Øvre Slottsgate 19,ved Max burger,59.912674,10.742025,False,30,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5217', 'i..."
2,5113,Leirfallsgata,Markveien 58,ved Markveien,59.919115,10.757831,False,15,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5113', 'i..."
3,4683,Valle Vision,Valle Vision,Valle,59.916065,10.807178,False,21,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/4683', 'i..."
4,4552,Bryn Brannstasjon,Agmund Bolts vei 4,Ved brannstasjonen,59.912716,10.813758,False,18,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/4552', 'i..."


In [57]:
from bysykkel_api import get_bike_info
availabiliy_info = get_bike_info(_headers)
availability_df = pd.DataFrame(availabiliy_info['data']['stations'])

In [58]:
print(availability_df.shape)
availability_df.head()

(251, 9)


,station_id,is_installed,is_renting,is_returning,last_reported,num_vehicles_available,num_bikes_available,num_docks_available,vehicle_types_available
0,5220,True,True,True,1733232716,0,0,18,"[{'vehicle_type_id': 'bike', 'count': 0}, {'ve..."
1,5217,True,True,True,1733232716,18,18,8,"[{'vehicle_type_id': 'bike', 'count': 18}, {'v..."
2,5113,True,True,True,1733232716,0,0,9,"[{'vehicle_type_id': 'bike', 'count': 0}, {'ve..."
3,4683,True,True,True,1733232716,0,0,20,"[{'vehicle_type_id': 'bike', 'count': 0}, {'ve..."
4,4552,True,True,True,1733232716,1,1,12,"[{'vehicle_type_id': 'bike', 'count': 1}, {'ve..."


In [ ]:
def match_stations_and_availability(stations_gdf, availability_df):
    stations_gdf = stations_gdf.merge(availability_df, left_on="station_id", right_on="station_id")
    return stations_gdf

stations_df = match_stations_and_availability(stations_gdf, availability_df)
stations_gdf = gpd.GeoDataFrame(stations_df, geometry=gpd.points_from_xy(stations_df.lon, stations_df.lat))
stations_gdf.head()

,station_id,name,address,cross_street,lat,lon,is_virtual_station,capacity,station_area,rental_uris,is_installed,is_renting,is_returning,last_reported,num_vehicles_available,num_bikes_available,num_docks_available,vehicle_types_available,geometry
0,5220,Blindern vgs,Sognsveien,Under broen,59.949881,10.730158,False,20,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5220', 'i...",True,True,True,1733232716,0,0,18,"[{'vehicle_type_id': 'bike', 'count': 0}, {'ve...",POINT (10.73016 59.94988)
1,5217,Egertorget,Øvre Slottsgate 19,ved Max burger,59.912674,10.742025,False,30,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5217', 'i...",True,True,True,1733232716,18,18,8,"[{'vehicle_type_id': 'bike', 'count': 18}, {'v...",POINT (10.74203 59.91267)
2,5113,Leirfallsgata,Markveien 58,ved Markveien,59.919115,10.757831,False,15,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/5113', 'i...",True,True,True,1733232716,0,0,9,"[{'vehicle_type_id': 'bike', 'count': 0}, {'ve...",POINT (10.75783 59.91911)
3,4683,Valle Vision,Valle Vision,Valle,59.916065,10.807178,False,21,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/4683', 'i...",True,True,True,1733232716,0,0,20,"[{'vehicle_type_id': 'bike', 'count': 0}, {'ve...",POINT (10.80718 59.91606)
4,4552,Bryn Brannstasjon,Agmund Bolts vei 4,Ved brannstasjonen,59.912716,10.813758,False,18,"{'type': 'MultiPolygon', 'coordinates': [[[[10...","{'android': 'oslobysykkel://stations/4552', 'i...",True,True,True,1733232716,1,1,12,"[{'vehicle_type_id': 'bike', 'count': 1}, {'ve...",POINT (10.81376 59.91272)


Now each row contains all the necessary information for the map, and analysis plus visualization can be done. The next step is to create the map witht the polygons for Oslo and the points for the stations. 

In [ ]:
points_in_polygons = gpd.sjoin(stations_gdf, arealer_gdf, how='inner')
polygon_point_count = points_in_polygons.groupby("index_right").size()

In [120]:
gdf_bydeler["point_count"] = gdf_bydeler.index.map(polygon_point_count).fillna(0)

In [121]:
gdf_bydeler

,fid,bydelsnr,bynavn,bydel,befolkning_2024,geometry,point_count
0,1,030101,Oslo,Gamle Oslo,63721,"MULTIPOLYGON (((10.66087 59.88365, 10.66472 59...",28.0
1,2,030102,Oslo,Grünerløkka,65532,"MULTIPOLYGON (((10.75378 59.91469, 10.75453 59...",47.0
2,3,030103,Oslo,Sagene,47627,"MULTIPOLYGON (((10.77156 59.93095, 10.77135 59...",17.0
3,4,030104,Oslo,St.Hanshaugen,41571,"MULTIPOLYGON (((10.75136 59.91923, 10.75158 59...",34.0
4,5,030105,Oslo,Frogner,60727,"MULTIPOLYGON (((10.65562 59.89038, 10.65381 59...",50.0
5,6,030106,Oslo,Ullern,35440,"MULTIPOLYGON (((10.66544 59.91574, 10.65614 59...",9.0
6,7,030107,Oslo,Vestre Aker,52590,"MULTIPOLYGON (((10.65615 59.93645, 10.65395 59...",6.0
7,8,030108,Oslo,Nordre Aker,55056,"MULTIPOLYGON (((10.78133 59.93899, 10.78102 59...",16.0
8,9,030109,Oslo,Bjerke,36450,"MULTIPOLYGON (((10.80766 59.92263, 10.80759 59...",1.0
9,10,030110,Oslo,Grorud,28080,"MULTIPOLYGON (((10.85822 59.94063, 10.85827 59...",0.0


In [ ]:
gdf_bydeler_non_empty = gdf_bydeler[gdf_bydeler["point_count"] > 0]

Notice Sentrum polygon is empty for population, highlighting which one it is to be kept in mind. Also showing the removal of the empty polygons. Adding the markers as well, to look at the more complete map.

In [64]:

bike_map = folium.Map(location=[59.91273, 10.74609], zoom_start=11)

folium.GeoJson(gdf_bydeler_non_empty).add_to(bike_map)
#Highligh where "bydel" is "Sentrum"
folium.GeoJson(gdf_bydeler_non_empty[gdf_bydeler_non_empty["bydel"] == "Sentrum"], style_function=lambda x: {"color": "red"}).add_to(bike_map)
bike_map
## Add the stations to the map, make the marker have number of available bikes from num_bikes_available_y column
from bysykkel_api import get_all_stations
from credentials import credentials

_headers = {
    "Client-Identifier": credentials["user_id"]
}


##response = get_all_stations(_headers)
#stations = response['data']['stations']
#oslo_coords = [59.9139, 10.7522]

def add_station_markers_with_pointers(map, stations_gdf):
    for _, station in stations_gdf.iterrows():
        # Create a folium.Icon with a dynamic number as the icon text
        icon_html = f"""
            <div style="
                font-size: 24px;  /* Adjust icon size */
                color: blue;      /* Adjust icon color */
                text-align: center;
                line-height: 1.2;">
                <i class="fa fa-map-marker" style="font-size: 36px;"></i><br>
                <span style="
                    color: white; 
                    font-size: 14px; 
                    font-weight: bold; 
                    position: relative;
                    top: -45px;">{station['num_bikes_available']}</span>
            </div>
        """
        icon = DivIcon(
            html=icon_html,
            icon_size=(30, 30)
        )
        # Add the marker to the map
        Marker(
            location=[station['lat'], station['lon']],
            popup=f"<b>{station['name']}</b><br>Available bikes: {station['num_bikes_available']}",
            icon=icon
        ).add_to(map)
add_station_markers_with_pointers(bike_map, stations_gdf)
bike_map
#save
bike_map.save("bike_map.html")

Create a method for the average amount of stations per population in each district.

In [65]:
def generate_stat_avg_(gdf_bydeler_non_empty)->list[tuple]:
    """Calculate the average habitants per station for each bydel

    Args:
        gdf_bydeler_non_empty (GeoDataFrame): Dataframe containing 'bydel' and 'befolkning_2024' columns

    Returns:
        list[tuple]:
         List of tuples containing the bydel and the average habitants per station
    """
    
    info = []
    for _, row in gdf_bydeler_non_empty.iterrows():
        # Calculate the average habitants per station
        bydel = row["bydel"]
        habitants_per_station = row["befolkning_2024"] / row["point_count"]
        # Append the tuple to the list
        info.append((bydel, habitants_per_station))
    return info

generate_stat_avg_(gdf_bydeler_non_empty)

[('Gamle Oslo', 2275.75),
 ('Grünerløkka', 1394.2978723404256),
 ('Sagene', 2801.5882352941176),
 ('St.Hanshaugen', 1222.6764705882354),
 ('Frogner', 1214.54),
 ('Ullern', 3937.777777777778),
 ('Vestre Aker', 8765.0),
 ('Nordre Aker', 3441.0),
 ('Bjerke', 36450.0),
 ('Alna', 50358.0),
 ('Sentrum', 0.0)]

In [ ]:
def generate_area_stat(gdf_bydeler_non_empty : GeoDataFrame) -> list[tuple]:
    """Calculate the average area per station for each bydel

    Args:
        gdf_bydeler_non_empty (GeoDataFrame): Dataframe containing 'bydel' and 'geometry' columns

    Returns:
        list[tuple]: List of tuples containing the bydel and the average area per station in 
    """
    info = []
    gdf_bydeler_non_empty = gdf_bydeler_non_empty.to_crs(epsg=25832)

    for _, row in gdf_bydeler_non_empty.iterrows():
        bydel = row["bydel"]
        area_per_station = (row["geometry"].area / 1e6) / row["point_count"]
        info.append((bydel, area_per_station))
    print(row["geometry"].area)
    return info


generate_area_stat(gdf_bydeler_non_empty)

[('Gamle Oslo', 0.5454755298781845),
 ('Grünerløkka', 0.1011538819207841),
 ('Sagene', 0.18287408225872007),
 ('St.Hanshaugen', 0.10517029341466054),
 ('Frogner', 0.28076238121039604),
 ('Ullern', 1.0856496899099866),
 ('Vestre Aker', 2.7607009277734282),
 ('Nordre Aker', 0.8554610710861222),
 ('Bjerke', 7.924520468082992),
 ('Alna', 13.693280650417606),
 ('Sentrum', 0.0631632207762691)]

In [67]:
gdf_bydeler_non_empty = gdf_bydeler_non_empty.to_crs(epsg=4326)
stations_gdf = stations_gdf.to_crs(epsg=4326)
stations_wide = gpd.sjoin(stations_gdf, gdf_bydeler_non_empty, how='inner')

def generate_bike_stat(stations_gdf : GeoDataFrame, gdf_bydeler) -> list[tuple]:
    """Calculate inhabitants per bike for each bydel

    Args:
        stations_gdf (GeoDataFrame): Dataframe containing 'bydel' and 'num_bikes_available' columns
        gdf_bydeler (GeoDataFrame): Dataframe containing 'bydel' and 'befolkning_2024' columns

    Returns:
        list[tuple]: List of tuples containing the bydel and the average bikes per station
    """

    info = []
    for _, row in gdf_bydeler.iterrows():
        bydel = row["bydel"]
        # Get the number of bikes in the bydel
        bikes_in_bydel = stations_gdf[stations_gdf["bydel"] == bydel]["num_bikes_available"].sum()
        # Calculate the inhabitants per bike
        inhabitants_per_bike = row["befolkning_2024"] / bikes_in_bydel
        if inhabitants_per_bike == float('inf'):
            inhabitants_per_bike = 0
        info.append((bydel, inhabitants_per_bike))
    return info

generate_bike_stat(stations_wide, gdf_bydeler_non_empty)

/tmp/ipykernel_535846/331882866.py:22: RuntimeWarning:

divide by zero encountered in scalar divide



[('Gamle Oslo', 413.77272727272725),
 ('Grünerløkka', 809.0370370370371),
 ('Sagene', 1322.9722222222222),
 ('St.Hanshaugen', 377.91818181818184),
 ('Frogner', 339.25698324022346),
 ('Ullern', 268.4848484848485),
 ('Vestre Aker', 3756.4285714285716),
 ('Nordre Aker', 917.6),
 ('Bjerke', 0),
 ('Alna', 50358.0),
 ('Sentrum', 0.0)]

In [68]:
def show_district_stat(bydel : str):
    """Show the statistics for a specific district

    Args:
        bydel (str): The district to show statistics for
    """
    gdf_bydeler_non_empty.to_crs(epsg=25832, inplace=True)
    
    district_info = gdf_bydeler_non_empty[gdf_bydeler_non_empty["bydel"] == bydel]
    stations_in_district = stations_wide[stations_wide["bydel"] == bydel]
    bikes_in_district = stations_in_district["num_bikes_available"].sum()
    habitants_in_district = district_info["befolkning_2024"].values[0]
    inhabitants_per_bike = habitants_in_district / bikes_in_district
    habitants_per_station = habitants_in_district / district_info["point_count"].values[0]
    area_per_station = (float(district_info["geometry"].area) / 1e6) / district_info["point_count"].values[0]
    print(f"District: {bydel}")
    print(float(district_info["geometry"].area))
    print(f"Number of bikes: {bikes_in_district}")
    print(f"Number of habitants: {habitants_in_district}")
    print(f"Inhabitants per bike: {inhabitants_per_bike}")
    print(f"Habitants per station: {habitants_per_station}")
    print(f"Average area per station: {area_per_station} km^2")

show_district_stat(bydel="Sentrum")

District: Sentrum
2652855.2726033325
Number of bikes: 527
Number of habitants: 0
Inhabitants per bike: 0.0
Habitants per station: 0.0
Average area per station: 0.06316322077626982 km^2


/tmp/ipykernel_535846/462210934.py:15: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead

/tmp/ipykernel_535846/462210934.py:17: FutureWarning:

Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead



In [ ]:
import plotly.graph_objects as go
import geopandas as gpd
import pandas as pd


fig = go.Figure()

gdf_bydeler_non_empty = gdf_bydeler_non_empty.to_crs(epsg=4326)
for _, feature in gdf_bydeler_non_empty.iterrows():
    geometry = feature['geometry']
    lon_list = []
    lat_list = []
    for poly in geometry.geoms:
        lon, lat = poly.exterior.xy
        lon_list.append(lon)
        lat_list.append(lat)
    
    fig.add_trace(go.Scattermapbox(
        lon=list(lon),  #
        lat=list(lat),  
        mode='lines',
        line=dict(color='blue', width=2),
        fill='toself',
        fillcolor='rgba(0, 128, 255, 0.1)',
        name=feature['bydel'],
        text = f"<b>{feature['bydel']}</b><br>Population: {feature['befolkning_2024']}",
        hoverinfo='text'
        
        ))

for _, station in stations_gdf.iterrows():
    fig.add_trace(go.Scattermapbox(
        lat=[station['lat']],
        lon=[station['lon']],
        mode='markers',
        marker=dict(
            size=12,
            color='red',
            opacity=0.8,
        ),
        text=f"<b>{station['name']}</b><br>Available bikes: {station['num_bikes_available']}",
        hoverinfo='text'
    ))

fig.update_layout(
    mapbox=dict(
        style="carto-positron",  # Choose your preferred Mapbox style
        center=dict(lat=59.91, lon=10.75),  # Center the map on Oslo
        zoom=11
    ),
    margin=dict(l=0, r=0, t=0, b=0),  # Remove extra margins
    height=600,
    title="Oslo Bike Stations and Districts",
    dragmode='zoom',
    showlegend=False,
)

fig.show()



In [94]:
print(type(lon_list))
station['lon']

<class 'list'>


10.776072